# Tagging genes with ddlite: candidate extraction

## Introduction
In this example **Snorkel** application, we'll build a toy gene tagger from scratch. Domain-specific tagging systems traditionally take months or years to develop. They use hand-crafted model circuitry and accurate, hand-labeled training data. We'll start to build a pretty good one in a few minutes with none of those things. The generalized extraction and learning utilities provided by **Snorkel** will allow us to turn a sampling of article abstracts and some basic domain knowledge into an automated tagging system. Specifically, we want an accurate tagger for genes in academic articles. We have comprehensive dictionaries of genes, but applying a simple matching rule might yield a lot of false positives. For example, "p53" might get tagged as a gene if it refers to a page number. Our goal is to use the [data programming](https://arxiv.org/abs/1605.07723) approach to improve precision.

Here's the pipeline we'll follow:

1. Obtain and parse input data (relevant article abstracts from PubMed)
2. Extract candidates for tagging
3. Generate features
4. Create a test set
5. Write labeling functions
6. Learn the tagging model
7. Iterate on labeling functions

Parts 1 and 2 are covered in this notebook, and parts 3 through 7 are covered in `GeneTaggerExample_Learning.ipynb`. Let's get to it.

In [26]:
%load_ext autoreload
%autoreload 2
import cPickle, os, sys

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Processing the input data
We already downloaded the raw HTML for 150 gene-related article pages from PubMed using the `pubmed_gene_html.py` script. These can be found in the `data` folder. We can use ddlite's `DocParser` to read in the article text. There's a general HTML parser which finds visible text, but we can do better by writing a more specific version to just grab the abstract text.

We'll use CoreNLP via ddlite's `SentenceParser` to parse each sentence. `DocParser` can handle this too; we didn't really need that call above. This can take a little while, so if the example has already been run, we'll reload it.

In [27]:
from snorkel.parser import HTMLDocParser, SentenceParser
from snorkel.parser import Corpus

class PubMedAbstractReader(HTMLDocParser):
    def _cleaner(self, s):
        return (s.parent.name == 'abstracttext')

doc_parser = PubMedAbstractReader('gene_tag_example/data/')

sent_parser = SentenceParser()

%time corpus = Corpus(doc_parser, sent_parser)

Parsing documents...
Parsing contexts...
CPU times: user 8.01 s, sys: 89.2 ms, total: 8.09 s
Wall time: 15.7 s


In [28]:
docs = list(corpus.get_docs())
print len(docs)
docs[0]

150


Document(id='6202659', file='6202659.html', text='In this study, we have correlated the hematological phenotype of 56 Sardinian beta o-thalassemia heterozygotes with their alpha-globin genotype as defined by restriction endonuclease mapping. We found that the coinheritance of the deletion of one alpha-globin and, more obviously, two alpha-globin genes tend to normalize the thalassemia-like hematological phenotype commonly associated with the beta o-thalassemia carrier state. On the other hand, the association of the deletion of three alpha-globin genes caused a more severe phenotype. By globin chain synthesis analysis, those beta o-thalassemia heterozygotes with the (-alpha/alpha alpha) alpha-globin genotype had less deficiency of beta-chain synthesis than did those with the normal alpha-globin genotype (alpha alpha/alpha alpha). In heterozygotes with the (-alpha/-alpha) and in those with the (--/-alpha) alpha-globin genotype the imbalance was actually reversed with a mild or marked al

In [29]:
sents = list(corpus.get_contexts())
print len(sents)
sents[0]

1225


Sentence(id='17217236-8', words=[u'We', u'analyzed', u'genotypes', u'and', u'alleles', u'of', u'the', u'5-HTTLPR', u'polymorphism', u'in', u'the', u'group', u'of', u'patients', u'and', u'in', u'the', u'control', u'group', u'.'], lemmas=[u'we', u'analyze', u'genotype', u'and', u'allele', u'of', u'the', u'5-httlpr', u'polymorphism', u'in', u'the', u'group', u'of', u'patient', u'and', u'in', u'the', u'control', u'group', u'.'], poses=[u'PRP', u'VBD', u'NNS', u'CC', u'NNS', u'IN', u'DT', u'NN', u'NN', u'IN', u'DT', u'NN', u'IN', u'NNS', u'CC', u'IN', u'DT', u'NN', u'NN', u'.'], dep_parents=[2, 0, 2, 3, 3, 9, 9, 9, 3, 12, 12, 9, 14, 12, 2, 19, 19, 19, 2, 2], dep_labels=[u'nsubj', u'ROOT', u'dobj', u'cc', u'conj', u'case', u'det', u'compound', u'nmod', u'case', u'det', u'nmod', u'case', u'nmod', u'cc', u'case', u'det', u'compound', u'conj', u'punct'], sent_id=8, doc_id='17217236', text=u'We analyzed genotypes and alleles of the 5-HTTLPR polymorphism in the group of patients and in the contro

## Extracting candidates with matchers
Extracting candidates for mentions (or relations) in ddlite is done with `Matcher` objects. First, we'll use a `DictionaryMatcher`. We have access to a pretty comprehensive gene dictionary. Let's load it in and create the `DictionaryMatcher`.

In [30]:
from snorkel.matchers import DictionaryMatch

# Schema is: ENSEMBL_ID | NAME | TYPE (refseq, canonical, non-canonical)
genes = [line.rstrip().split('\t')[1] for line in open('gene_tag_example/dicts/ensembl_genes.tsv')]
genes = filter(lambda g : len(g) > 2, genes)

gene_dm = DictionaryMatch(d=genes, ignore_case=False)

The dictionary match should provide fairly high recall, but we may still miss some candidates. We know that gene names are named nouns and are often all uppercase. Let's use DDLite's *compositional* matcher operations to handle this. First, we'll write a matcher to find all nouns using the parts-of-speech tags. Then, we'll use a filter to find uppercase sequences. Finally, we'll use a filter to make sure each match has at least 3 characters. We pass `noun_rm` to `up_rm`, and `up_rm` to the final filter to compose them with each other.

In [31]:
from snorkel.matchers import RegexMatchSpan, RegexMatchEach

noun_regex  = RegexMatchEach(rgx=r'NN.*', attrib='poses')
up_regex    = RegexMatchSpan(noun_regex, rgx=r'[A-Z]+([0-9]+)?([A-Z]+)?([0-9]+)?$', ignore_case=False, attrib='words')
multi_regex = RegexMatchSpan(up_regex, rgx=r'[a-z0-9]{3,}', ignore_case=True, attrib='words')

Now, we put it all together using the `Union` matcher, and also define the base `CandidateSpace`--in this case, just all n-grams of length 3 or less, using `Ngrams`:

In [32]:
from snorkel.matchers import Union
from snorkel.candidates import Ngrams

matcher = Union(gene_dm, multi_regex)

ngrams = Ngrams(n_max=3)

Finally, we'll use the `Candidates` object to run the matcher over the candidate space, and hold the resulting set of `Candidate` objects (which in our case will be the `Ngram` subclass).  Note that we can run this process in parallel by setting the `parallelism` value (however, for a small set such as the one here, this may introduce more overhead processing time than it saves with parallelism):

In [ ]:
from snorkel.candidates import Candidates

%time candidates = Candidates(ngrams, matcher, sents, parallelism=False)
print len(list(candidates))
list(candidates)[:5]

Extracting candidates...
CPU times: user 2.91 s, sys: 163 ms, total: 3.07 s
Wall time: 2.96 s
1574


[<Ngram("SBH1", id=12665530-1:136-139, chars=[136,139], words=[4,4]),
 <Ngram("CAD", id=24508273-5:658-660, chars=[658,660], words=[23,23]),
 <Ngram("ADHD", id=24132904-5:1300-1303, chars=[1300,1303], words=[24,24]),
 <Ngram("MODY", id=16035308-1:244-247, chars=[244,247], words=[0,0]),
 <Ngram("IPF", id=16035308-2:486-488, chars=[486,488], words=[30,30])]

Finally, we'll just pickle this `Candidates` object to save it for the next notebook:

In [ ]:
with open('gene_tag_example/gene_tag_saved_candidates.pkl', 'w+') as f:
    cPickle.dump(candidates, f)